In [ ]:
#numpy 前后神经网络

# -*- coding: utf-8 -*-
import numpy as np

# N是批量大小; D_in是输入维度;
# 49/5000 H是隐藏的维度; D_out是输出维度。
N, D_in, H, D_out = 64, 1000, 100, 10

# 创建随机输入和输出数据
x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

# 随机初始化权重
w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)



learning_rate = 1e-6
for t in range(500):
    # 前向传递：计算预测值y
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)

    # 计算和打印损失loss
    loss = np.square(y_pred - y).sum()
    print(t, loss)

    # 反向传播，计算w1和w2对loss的梯度
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)

    # 更新权重
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2


In [ ]:
# -*- coding: utf-8 -*-


"""
张量（Tensor）：PyTorch的tensor在概念上与numpy的array相同： 
tensor是一个n维数组，PyTorch提供了许多函数用于操作这些张量。任何希望使用NumPy执行的计算也可以使用PyTorch的tensor来完成，
可以认为它们是科学计算的通用工具。

与Numpy不同，PyTorch可以利用GPU加速其数值计算。要在GPU上运行Tensor,在构造张量使用device参数把tensor建立在GPU上。 
"""

import torch


dtype = torch.float
device = torch.device("cpu")
# device = torch.device（“cuda：0”）＃取消注释以在GPU上运行

# N是批量大小; D_in是输入维度;
# H是隐藏的维度; D_out是输出维度。
N, D_in, H, D_out = 64, 1000, 100, 10

#创建随机输入和输出数据
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 随机初始化权重
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)



learning_rate = 1e-6
for t in range(500):
    # 前向传递：计算预测y
    h = x.mm(w1)    #torch类型的点乘
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)

    # 计算和打印损失
    loss = (y_pred - y).pow(2).sum().item()
    print(t, loss)

    # Backprop计算w1和w2相对于损耗的梯度
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)

    # 使用梯度下降更新权重
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

In [1]:
# -*- coding: utf-8 -*-
import torch

dtype = torch.float
device = torch.device("cpu")
# device = torch.device（“cuda：0”）＃取消注释以在GPU上运行

# N是批量大小; D_in是输入维度;
# H是隐藏的维度; D_out是输出维度。
N, D_in, H, D_out = 64, 1000, 100, 10

# 创建随机Tensors以保持输入和输出。
# 设置requires_grad = False表示我们不需要计算渐变
# 在向后传球期间对于这些Tensors。
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 为权重创建随机Tensors。
# 设置requires_grad = True表示我们想要计算渐变
# 在向后传球期间尊重这些张贴。
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # 前向传播：使用tensors上的操作计算预测值y; 
      # 由于w1和w2有requires_grad=True，涉及这些张量的操作将让PyTorch构建计算图，
    # 从而允许自动计算梯度。由于我们不再手工实现反向传播，所以不需要保留中间值的引用。
    y_pred = x.mm(w1).clamp(min=0).mm(w2)

    # 使用Tensors上的操作计算和打印丢失。
    # loss是一个形状为()的张量
    # loss.item() 得到这个张量对应的python数值
    loss = (y_pred - y).pow(2).sum()
    print(t, loss.item())

    # 使用autograd计算反向传播。这个调用将计算loss对所有requires_grad=True的tensor的梯度。
    # 这次调用后，w1.grad和w2.grad将分别是loss对w1和w2的梯度张量。
    loss.backward()

    # 使用梯度下降更新权重。对于这一步，我们只想对w1和w2的值进行原地改变；不想为更新阶段构建计算图，
    # 所以我们使用torch.no_grad()上下文管理器防止PyTorch为更新构建计算图
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # 反向传播后手动将梯度设置为零
        w1.grad.zero_()
        w2.grad.zero_()

0 26708958.0
1 19818612.0
2 18632984.0
3 20466468.0
4 23426750.0
5 25367762.0
6 23909830.0
7 18945362.0
8 12447779.0
9 7168574.0
10 3877202.5
11 2146454.75
12 1286013.125
13 856368.6875
14 628188.1875
15 495268.0625
16 409016.3125
17 347395.15625
18 300063.375
19 261965.25
20 230340.25
21 203622.421875
22 180792.71875
23 161125.875
24 144085.796875
25 129210.390625
26 116176.2890625
27 104707.4765625
28 94573.5625
29 85606.59375
30 77643.7265625
31 70536.9375
32 64194.91796875
33 58519.75
34 53430.9609375
35 48857.8046875
36 44737.64453125
37 41018.421875
38 37657.359375
39 34613.1328125
40 31852.20703125
41 29346.6328125
42 27068.00390625
43 24991.4765625
44 23098.998046875
45 21368.392578125
46 19784.53515625
47 18334.05078125
48 17003.171875
49 15780.541015625
50 14656.783203125
51 13622.5205078125
52 12669.720703125
53 11791.6572265625
54 10981.5703125
55 10233.2724609375
56 9541.435546875
57 8901.4931640625
58 8309.021484375
59 7760.15234375
60 7251.263671875
61 6779.3271484375
62

442 0.00047864069347269833
443 0.0004668429319281131
444 0.0004550816083792597
445 0.00044408865505829453
446 0.0004348561051301658
447 0.0004241667629685253
448 0.0004145086568314582
449 0.0004053988086525351
450 0.0003962661139667034
451 0.00038758799200877547
452 0.00037904217606410384
453 0.0003693429462146014
454 0.0003613020235206932
455 0.00035353098064661026
456 0.00034559896448627114
457 0.00033805702696554363
458 0.0003310505417175591
459 0.0003232513554394245
460 0.00031684068380855024
461 0.00030962988967075944
462 0.00030325594707392156
463 0.00029770610854029655
464 0.00029126848676241934
465 0.0002862136752810329
466 0.0002796488697640598
467 0.00027389617753215134
468 0.0002683783241081983
469 0.00026308302767574787
470 0.00025745161110535264
471 0.00025259278481826186
472 0.0002476144290994853
473 0.0002422044926788658
474 0.00023759121540933847
475 0.00023299964959733188
476 0.00022875447757542133
477 0.00022380921291187406
478 0.00021959812147542834
479 0.00021515700